In [1]:
import time
import numpy as np
import pandas as pd
import keras
from keras.engine.topology import Layer

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
def mape_loss_func(preds, labels):
    mask = labels > 5
    return np.mean(np.fabs(labels[mask]-preds[mask])/labels[mask])

def smape_loss_func(preds, labels):
    mask= labels > 5
    return np.mean(2*np.fabs(labels[mask]-preds[mask])/(np.fabs(labels[mask])+np.fabs(preds[mask])))

def mae_loss_func(preds, labels):
    mask= labels > 5
    return np.fabs((labels[mask]-preds[mask])).mean()

def eliminate_nan(b):
    a = np.array(b)
    c = a[~np.isnan(a)]
    return c

In [3]:
#载入最近路段数据，流量数据
time3 = time.time()

# distance = np.array(pd.read_csv('video_link_dis.csv',header = None))
near_road = np.array(pd.read_csv('./data/network/2small_network_nearest_road_id.csv',header = 0))
flow = np.array(pd.read_csv("./data/network/2small_network_speed.csv", header= 0)) #注意header=0 or None

In [4]:
# 利用滑动窗口的方式，重构数据为(n，最近路段数，输入时间窗，总路段数)的形式

k = 5 # 参数k为需考虑的最近路段数
t_p = 24 # 参数t_p为总时间序列长度（天）
t_input = 12 #参数t_input为输入时间窗(5min颗粒度)
t_pre = 3 #参数t_pre为预测时间窗(5min颗粒度)
num_links = 30 #参数num_links为总路段数


image = []
for i in range(np.shape(near_road)[0]):
    road_id = []
    for j in range(k):
        road_id.append(near_road[i][j])
    image.append(flow[road_id, :])
image1 = np.reshape(image, [-1, k, len(flow[0,:])])
image2 = np.transpose(image1,(1,2,0))
image3 = []
label = []
day = []

for i in range(1,t_p):
    for j in range(180-t_input-t_pre):
        image3.append(image2[:, i*180+j:i*180+j+t_input, :][:])
        label.append(flow[:, i*180+j+t_input:i*180+j+t_input+t_pre][:])
        day.append(flow[:, (i-1)*180+j+t_input:(i-1)*180+j+t_input+t_pre][:])

image3 = np.asarray(image3)
label = np.asarray(label)
day =  np.asarray(day)

print(np.shape(image3))
print(np.shape(label))
print(np.shape(day))

#划分前80%数据为训练集，最后20%数据为测试集
image_train = image3[:np.shape(image3)[0]*4//5]
image_test = image3[np.shape(image3)[0]*4//5:]
label_train = label[:np.shape(label)[0]*4//5]
label_test = label[np.shape(label)[0]*4//5:]

day_train = day[:np.shape(day)[0]*4//5]
day_test = day[np.shape(day)[0]*4//5:]


time4 = time.time()
print('input done %g' % (time4-time3))

(3795, 5, 12, 30)
(3795, 30, 3)
(3795, 30, 3)
input done 0.259437


In [5]:
print(np.shape(image1))

(30, 5, 5222)


In [6]:
print(np.shape(image2))

(5, 5222, 30)


In [7]:
# define merge layer
class Merge_Layer(Layer):
    def __init__(self, **kwargs):
        super(Merge_Layer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.para1 = self.add_weight(shape=(input_shape[0][1], input_shape[0][2]),
                                     initializer='uniform', trainable=True,
                                     name='para1')
        self.para2 = self.add_weight(shape=(input_shape[1][1], input_shape[1][2]),
                                     initializer='uniform', trainable=True,
                                     name='para2')
        super(Merge_Layer, self).build(input_shape)

    def call(self, inputs):
        mat1 = inputs[0]
        mat2 = inputs[1]
        output = mat1 * self.para1 + mat2 * self.para2
        # output = mat1 * 0.1 + mat2 * 0.9
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [8]:
input_data = keras.Input(shape=(k,t_input,num_links), name='input_data')
input_HA = keras.Input(shape=(num_links, t_pre), name='input_HA')

x = keras.layers.BatchNormalization(input_shape =(k,t_input,num_links))(input_data)

x = keras.layers.Conv2D(
                           filters = 30,
                           kernel_size = 3,
                           strides = 1,
                           padding="SAME",
                           activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)

x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(
                       filters = 30,
                       kernel_size = 3,
                       strides = 1,
                       padding="SAME",
                       activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)
x = keras.layers.Flatten()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)

x = keras.layers.Dense(num_links*2*t_pre, activation='relu', name='dense_1')(x)
x = keras.layers.Dense(num_links*t_pre, activation='relu', name='dense_2')(x)

output = keras.layers.Reshape((num_links,t_pre))(x)

output_final = Merge_Layer()([output, input_HA])

# construct model
finish_model = keras.models.Model([input_data,input_HA], [output_final])

finish_model.summary()








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         (None, 5, 12, 30)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 5, 12, 30)    120         input_data[0][0]                 
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 5, 12, 30)    8130        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
average_pooling2d_1 (AveragePoo (None, 5, 12, 30)    0           conv2d_1[0][0]       

In [9]:
finish_model.compile(optimizer='adam', loss='mean_squared_error')

In [10]:
X_train = image_train
X_HA_train = day_train
label_train = label_train

In [13]:
#模型拟合与评估
finish_model.fit([X_train,X_HA_train], label_train, epochs=100, batch_size=128,
validation_data=([image_test,day_test], label_test))
# finish_model.evaluate(image_test, label_test)

Train on 3036 samples, validate on 759 samples
Epoch 1/30
3036/3036 [==============================] - 0s 92us/step - loss: 10.8963 - val_loss: 11.0085
Epoch 2/30
3036/3036 [==============================] - 0s 81us/step - loss: 10.7361 - val_loss: 11.0460
Epoch 3/30
3036/3036 [==============================] - 0s 82us/step - loss: 10.6501 - val_loss: 10.7159
Epoch 4/30
3036/3036 [==============================] - 0s 81us/step - loss: 10.4701 - val_loss: 10.6054
Epoch 5/30
3036/3036 [==============================] - 0s 80us/step - loss: 10.3718 - val_loss: 10.5115
Epoch 6/30
3036/3036 [==============================] - 0s 82us/step - loss: 10.3401 - val_loss: 10.4071
Epoch 7/30
3036/3036 [==============================] - 0s 81us/step - loss: 10.3201 - val_loss: 10.4730
Epoch 8/30
3036/3036 [==============================] - 0s 84us/step - loss: 10.1830 - val_loss: 10.1376
Epoch 9/30
3036/3036 [==============================] - 0s 79us/step - loss: 10.0993 - val_loss: 10.0528
Epoch 10

In [14]:
#模型预测
model_pre = finish_model.predict([image_test,day_test])

In [15]:
#计算各项误差指标

mape_mean = mape_loss_func(model_pre, label_test)
smape_mean = smape_loss_func(model_pre, label_test)
mae_mean = mae_loss_func(model_pre, label_test)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.11549594534118575
smape = 0.11479005268693586
mae = 2.289657575671831


In [16]:
#模型保存
finish_model.save_weights('./model/source.h5')

In [17]:
#计算每条路段的误差
mape_list = []
for i in range(num_links):
    a1 = mape_loss_func(model_pre[:,i,:], label_test[:,i,:])
    mape_list.append(a1)
    print(str(i+1)+'th link')
    print(a1)

1th link
0.10760044966308141
2th link
0.09500663733865604
3th link
0.08070501714913837
4th link
0.09171510411981672
5th link
0.10872764727592654
6th link
0.08875469993807626
7th link
0.12791570212879105
8th link
0.11264537376763568
9th link
0.12379884839738238
10th link
0.11831060756296126
11th link
0.09528742675340195
12th link
0.1361543984353815
13th link
0.12167354815597616
14th link
0.09876509721440604
15th link
0.12634616772590745
16th link
0.11137772896708985
17th link
0.11459936123306677
18th link
0.12389949078817748
19th link
0.12538358921853604
20th link
0.16122264393426863
21th link
0.11079864508247046
22th link
0.10832288690204786
23th link
0.11263145017031183
24th link
0.1094690772598296
25th link
0.10891831729766785
26th link
0.10422287540571536
27th link
0.13873103228550387
28th link
0.12804934401273513
29th link
0.15456284353067223
30th link
0.11928234852094047


In [18]:
mape_pd = pd.Series(mape_list)

In [19]:
mape_pd.sort_values()

2     0.080705
5     0.088755
3     0.091715
1     0.095007
10    0.095287
13    0.098765
25    0.104223
0     0.107600
21    0.108323
4     0.108728
24    0.108918
23    0.109469
20    0.110799
15    0.111378
22    0.112631
7     0.112645
16    0.114599
9     0.118311
29    0.119282
12    0.121674
8     0.123799
17    0.123899
18    0.125384
14    0.126346
6     0.127916
27    0.128049
11    0.136154
26    0.138731
28    0.154563
19    0.161223
dtype: float64